# 다양한 데이터 타입 다루기

In [0]:
from pyspark.sql import functions as F
from pyspark.sql import types as T

In [0]:
path= '/FileStore/tables/2010_12_01.csv'

In [0]:
df = spark.read.format('csv').option('header', 'true').option('inferSchema', 'true').load(path)

In [0]:
df.printSchema()

root
-- InvoiceNo: string (nullable = true)
-- StockCode: string (nullable = true)
-- Description: string (nullable = true)
-- Quantity: integer (nullable = true)
-- InvoiceDate: string (nullable = true)
-- UnitPrice: double (nullable = true)
-- CustomerID: double (nullable = true)
-- Country: string (nullable = true)

In [0]:
df.createOrReplaceTempView

Out[5]: <bound method DataFrame.createOrReplaceTempView of DataFrame[InvoiceNo: string, StockCode: string, Description: string, Quantity: int, InvoiceDate: string, UnitPrice: double, CustomerID: double, Country: string]>

In [0]:
df.show(3)

+---------+---------+--------------------+--------+-------------------+---------+----------+--------------+
InvoiceNo|StockCode| Description|Quantity| InvoiceDate|UnitPrice|CustomerID| Country|
+---------+---------+--------------------+--------+-------------------+---------+----------+--------------+
 536365| 85123A|WHITE HANGING HEA...| 6|2010-12-01 08:26:00| 2.55| 17850.0|United Kingdom|
 536365| 71053| WHITE METAL LANTERN| 6|2010-12-01 08:26:00| 3.39| 17850.0|United Kingdom|
 536365| 84406B|CREAM CUPID HEART...| 8|2010-12-01 08:26:00| 2.75| 17850.0|United Kingdom|
+---------+---------+--------------------+--------+-------------------+---------+----------+--------------+
only showing top 3 rows

## 스파크 데이터 타입으로 변환하기
- lit함수를 적용하여 다른언어의 데이터 타입을 스파크 데이터 타입에 맞게 변환

In [0]:
df.select(F.lit(5), F.lit('five'), F.lit(5.0)) #int, string, double

Out[7]: DataFrame[5: int, five: string, 5.0: double]

## 불리언 데이터 타입 다루기
- 불리언은 모든 필터링 작업의 기반
- 불리언 구문은 and, or, true, false로 구성
- 불리언 구문을 사용해 true 또는 false로 평가되는 논리 문법을 만듦
- 불리언 식에는 일치 조건, 비교 연산 조건을 사용 가능

In [0]:
df.where(F.col('invoiceno') != 536365).select('invoiceno', 'description').show(5, False)

+---------+-----------------------------+
invoiceno|description |
+---------+-----------------------------+
536366 |HAND WARMER UNION JACK |
536366 |HAND WARMER RED POLKA DOT |
536367 |ASSORTED COLOUR BIRD ORNAMENT|
536367 |POPPY'S PLAYHOUSE BEDROOM |
536367 |POPPY'S PLAYHOUSE KITCHEN |
+---------+-----------------------------+
only showing top 5 rows

In [0]:
priceFilter = F.col('unitprice')>600
descripFilter = F.instr(df.Description, 'POSTAGE')>=1 #POSTAGE가 없으면 0 있으면 1부터(index가 1부터임)
df.where(df.StockCode.isin('DOT')).where(priceFilter|descripFilter).show() #isin메서드로 DOT값이 있는지 확인

+---------+---------+--------------+--------+-------------------+---------+----------+--------------+
InvoiceNo|StockCode| Description|Quantity| InvoiceDate|UnitPrice|CustomerID| Country|
+---------+---------+--------------+--------+-------------------+---------+----------+--------------+
 536544| DOT|DOTCOM POSTAGE| 1|2010-12-01 14:32:00| 569.77| null|United Kingdom|
 536592| DOT|DOTCOM POSTAGE| 1|2010-12-01 17:06:00| 607.49| null|United Kingdom|
+---------+---------+--------------+--------+-------------------+---------+----------+--------------+

-----
- [instr_링크](https://spark.apache.org/docs/latest/api/python/reference/api/pyspark.sql.functions.instr.html?highlight=instr#pyspark.sql.functions.instr)
- [isin 링크](https://spark.apache.org/docs/latest/api/python/reference/api/pyspark.sql.Column.isin.html?highlight=isin#pyspark.sql.Column.isin)

## 수치형 데이터 타입 다루기
- pow: 거듭제곱
- round: 반올림
- bound: 내림
- corr: 피어슨 상관계수
- describe: 요약 통계 계산
  - count
  - mean
  - stddev
  - min
  - max

In [0]:
fabricatedQuantity = F.pow(F.col('Quantity') * F.col('UnitPrice'), 2)+5
df.select('customerid', fabricatedQuantity.alias('realQuantity')).show(2)

+----------+------------------+
customerid| realQuantity|
+----------+------------------+
 17850.0|239.08999999999997|
 17850.0| 418.7156|
+----------+------------------+
only showing top 2 rows

In [0]:
df.select(F.round(F.lit("2.5")), F.bround(F.lit("2.5")))

Out[11]: DataFrame[round(2.5, 0): double, bround(2.5, 0): double]

In [0]:
df.select(F.corr('Quantity', 'UnitPrice')).show()

+-------------------------+
corr(Quantity, UnitPrice)|
+-------------------------+
 -0.04112314436835551|
+-------------------------+

In [0]:
df.describe().show()

+-------+-----------------+------------------+--------------------+------------------+-------------------+------------------+------------------+--------------+
summary| InvoiceNo| StockCode| Description| Quantity| InvoiceDate| UnitPrice| CustomerID| Country|
+-------+-----------------+------------------+--------------------+------------------+-------------------+------------------+------------------+--------------+
 count| 3108| 3108| 3098| 3108| 3108| 3108| 1968| 3108|
 mean| 536516.684944841|27834.304044117645| null| 8.627413127413128| null| 4.151946589446603|15661.388719512195| null|
 stddev|72.89447869788873|17407.897548583845| null|26.371821677029203| null|15.638659854603892|1854.4496996893627| null|
 min| 536365| 10002| 4 PURPLE FLOCK D...| -24|2010-12-01 08:26:00| 0.0| 12431.0| Australia|
 max| C536548| POST|ZINC WILLIE WINKI...| 600|2010-12-01 17:35:00| 607.49| 18229.0|United Kingdom|
+-------+-----------------+------------------+--------------------+------------------+-------------------+------------------+------------------+--------------+

#### StatFunctions 패키지
- [doc 링크](https://spark.apache.org/docs/latest/api/python/reference/api/pyspark.sql.DataFrameStatFunctions.html?highlight=statfunctions#pyspark.sql.DataFrameStatFunctions)
- 다양한 통곗값을 계산할 때 사용하는 DataFrame 메서드
- <strong>stat 속성</strong>을 사용해 접근할 수 있음

In [0]:
olName= 'UnitPrice'
quantileProbs = [0.5]
relError=0.05

#백분위수를 정확하게 계산하거나 근사치 계산
df.stat.approxQuantile(olName, quantileProbs, relError)

Out[14]: [2.51]

In [0]:
#피어슨 상관계수
df.stat.corr('Quantity', 'UnitPrice')

Out[15]: -0.04112314436835551

In [0]:
#crosstab: col1과 col2의 교차표(두 변수의 범주별 조합빈도수를 작성한 표)
display(df.stat.crosstab('StockCode', 'Quantity').limit(10))

StockCode_Quantity,-1,-10,-12,-2,-24,-3,-4,-5,-6,-7,1,10,100,11,12,120,128,13,14,144,15,16,17,18,19,192,2,20,200,21,216,22,23,24,25,252,27,28,288,3,30,32,33,34,36,384,4,40,432,47,48,480,5,50,56,6,60,600,64,7,70,72,8,80,9,96
22578,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
21327,0,0,0,0,0,0,0,0,0,0,2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
22064,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
21080,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
22219,0,0,0,0,0,0,0,0,0,0,0,0,0,0,3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
21908,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
22818,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0
15056BL,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0
72817,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
22545,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [0]:
#모든 로우에 고유 ID 추가
df.select(F.monotonically_increasing_id()).show(5)

+-----------------------------+
monotonically_increasing_id()|
+-----------------------------+
 0|
 1|
 2|
 3|
 4|
+-----------------------------+
only showing top 5 rows

## 문자열 데이터 타입 다루기
- 데이터 추출
- 데이터 치환
- 문자열 존재 여부
- 대/소문자 변환 처리
- etc.

In [0]:
display(df.select('Description').limit(10))

Description
WHITE HANGING HEART T-LIGHT HOLDER
WHITE METAL LANTERN
CREAM CUPID HEARTS COAT HANGER
KNITTED UNION FLAG HOT WATER BOTTLE
RED WOOLLY HOTTIE WHITE HEART.
SET 7 BABUSHKA NESTING BOXES
GLASS STAR FROSTED T-LIGHT HOLDER
HAND WARMER UNION JACK
HAND WARMER RED POLKA DOT
ASSORTED COLOUR BIRD ORNAMENT


In [0]:
#각 문자열을 공백 기준으로 나눴을 때, 모든 첫 글자를 대문자로 변경
display(df.select(F.initcap(F.col('Description'))).limit(10))

initcap(Description)
White Hanging Heart T-light Holder
White Metal Lantern
Cream Cupid Hearts Coat Hanger
Knitted Union Flag Hot Water Bottle
Red Woolly Hottie White Heart.
Set 7 Babushka Nesting Boxes
Glass Star Frosted T-light Holder
Hand Warmer Union Jack
Hand Warmer Red Polka Dot
Assorted Colour Bird Ornament


In [0]:
#소문자, 대문자로 변환
display(df.select(F.lower('Description'), F.upper('Description')).limit(10))

lower(Description),upper(Description)
white hanging heart t-light holder,WHITE HANGING HEART T-LIGHT HOLDER
white metal lantern,WHITE METAL LANTERN
cream cupid hearts coat hanger,CREAM CUPID HEARTS COAT HANGER
knitted union flag hot water bottle,KNITTED UNION FLAG HOT WATER BOTTLE
red woolly hottie white heart.,RED WOOLLY HOTTIE WHITE HEART.
set 7 babushka nesting boxes,SET 7 BABUSHKA NESTING BOXES
glass star frosted t-light holder,GLASS STAR FROSTED T-LIGHT HOLDER
hand warmer union jack,HAND WARMER UNION JACK
hand warmer red polka dot,HAND WARMER RED POLKA DOT
assorted colour bird ornament,ASSORTED COLOUR BIRD ORNAMENT


In [0]:
#문자열 주변 공백 제거 및 추가
target= '     HELLO     '
display(df.select(F.trim(F.lit(target)), F.ltrim(F.lit(target)), F.rtrim(F.lit(target)), F.rpad(F.lit('HELLO'),10, '-'), F.lpad(F.lit('HELLO'),10,'-')))

trim( HELLO ),ltrim( HELLO ),rtrim( HELLO ),"rpad(HELLO, 10, -)","lpad(HELLO, 10, -)"
HELLO,HELLO,HELLO,HELLO-----,-----HELLO
HELLO,HELLO,HELLO,HELLO-----,-----HELLO
HELLO,HELLO,HELLO,HELLO-----,-----HELLO
HELLO,HELLO,HELLO,HELLO-----,-----HELLO
HELLO,HELLO,HELLO,HELLO-----,-----HELLO
HELLO,HELLO,HELLO,HELLO-----,-----HELLO
HELLO,HELLO,HELLO,HELLO-----,-----HELLO
HELLO,HELLO,HELLO,HELLO-----,-----HELLO
HELLO,HELLO,HELLO,HELLO-----,-----HELLO
HELLO,HELLO,HELLO,HELLO-----,-----HELLO


### 정규 표현식

In [0]:
#regex_string에서 정의한 색과 관련된 단어 모두 COLOR로 치환
regex_string="BLACK|WHITE|RED|GREEN|BLUE"
display(df.select(F.regexp_replace(F.col('description'), regex_string, 'COLOR').alias('color_clean'),F.col('description')).limit(10))

color_clean,description
COLOR HANGING HEART T-LIGHT HOLDER,WHITE HANGING HEART T-LIGHT HOLDER
COLOR METAL LANTERN,WHITE METAL LANTERN
CREAM CUPID HEARTS COAT HANGER,CREAM CUPID HEARTS COAT HANGER
KNITTED UNION FLAG HOT WATER BOTTLE,KNITTED UNION FLAG HOT WATER BOTTLE
COLOR WOOLLY HOTTIE COLOR HEART.,RED WOOLLY HOTTIE WHITE HEART.
SET 7 BABUSHKA NESTING BOXES,SET 7 BABUSHKA NESTING BOXES
GLASS STAR FROSTED T-LIGHT HOLDER,GLASS STAR FROSTED T-LIGHT HOLDER
HAND WARMER UNION JACK,HAND WARMER UNION JACK
HAND WARMER COLOR POLKA DOT,HAND WARMER RED POLKA DOT
ASSORTED COLOUR BIRD ORNAMENT,ASSORTED COLOUR BIRD ORNAMENT


In [0]:
#정규 표현식을 만들지 않고 간단하게 문자 단위로 치환(L=1, E=3, T=7)
display(df.select(F.translate(F.col('description'), 'LEET' , '1337'), F.col('description')).limit(5))

"translate(description, LEET, 1337)",description
WHI73 HANGING H3AR7 7-1IGH7 HO1D3R,WHITE HANGING HEART T-LIGHT HOLDER
WHI73 M37A1 1AN73RN,WHITE METAL LANTERN
CR3AM CUPID H3AR7S COA7 HANG3R,CREAM CUPID HEARTS COAT HANGER
KNI773D UNION F1AG HO7 WA73R BO7713,KNITTED UNION FLAG HOT WATER BOTTLE
R3D WOO11Y HO77I3 WHI73 H3AR7.,RED WOOLLY HOTTIE WHITE HEART.


In [0]:
# 정규식에 매칭되는 그룹 추출 (인덱스 지정 -> 1로하면 정규식에 처음으로 매칭되는 문자열을 추출)
extract_str = "(BLACK|WHITE|RED|GREEN|BLUE)"
display(df.select(F.regexp_extract(F.col('description'), extract_str, 1).alias('color_clean'), F.col('description')).limit(5))

color_clean,description
WHITE,WHITE HANGING HEART T-LIGHT HOLDER
WHITE,WHITE METAL LANTERN
,CREAM CUPID HEARTS COAT HANGER
,KNITTED UNION FLAG HOT WATER BOTTLE
RED,RED WOOLLY HOTTIE WHITE HEART.


In [0]:
#값 추출 없이 단순히 값의 존재 여부를 확인하고 싶을 때
containsBlack = F.instr(F.col('description'), 'BLACK') >=1
containsWhite= F.instr(F.col('description'), 'WHITE') >=1

display(df.withColumn('hasSimpleColor', containsBlack | containsWhite)\
.where('hasSimpleColor').select('description').limit(5))


description
WHITE HANGING HEART T-LIGHT HOLDER
WHITE METAL LANTERN
RED WOOLLY HOTTIE WHITE HEART.
WHITE HANGING HEART T-LIGHT HOLDER
WHITE METAL LANTERN


------
- 위에는 값을 두 개만 사용해서 간단하지만 값의 개수가 늘어난다면 복잡해질 것임

In [0]:
#값의 개수가 늘어났을 때 해결 방법
simpleColors = ['black', 'white','red', 'green', 'blue']
def color_locator(col, color_string):
  #주어진 문자열(substr)과 처음으로 일치하는 위치를 반환하는 locate함수 -> 일치하는 부분이 없으면 0을 반환
  return F.locate(color_string.upper(), col).cast(T.BooleanType()).alias('is_'+color_string)

selectedCol = [color_locator(df['description'], c) for c in simpleColors]
selectedCol.append(F.expr('*'))

In [0]:
selectedCol

Out[27]: [Column<'CAST(locate(BLACK, description, 1) AS BOOLEAN) AS `is_black`'>,
 Column<'CAST(locate(WHITE, description, 1) AS BOOLEAN) AS `is_white`'>,
 Column<'CAST(locate(RED, description, 1) AS BOOLEAN) AS `is_red`'>,
 Column<'CAST(locate(GREEN, description, 1) AS BOOLEAN) AS `is_green`'>,
 Column<'CAST(locate(BLUE, description, 1) AS BOOLEAN) AS `is_blue`'>,
 Column<'unresolvedstar()'>]

In [0]:
display(df.select(*selectedCol).where(F.expr('is_white or is_red')).select('description').limit(10))

description
WHITE HANGING HEART T-LIGHT HOLDER
WHITE METAL LANTERN
RED WOOLLY HOTTIE WHITE HEART.
HAND WARMER RED POLKA DOT
RED COAT RACK PARIS FASHION
ALARM CLOCK BAKELIKE RED
SET/2 RED RETROSPOT TEA TOWELS
RED TOADSTOOL LED NIGHT LIGHT
HAND WARMER RED POLKA DOT
WHITE HANGING HEART T-LIGHT HOLDER


## 날짜와 타임스탬프 데이터 타입 다루기
- 스파크는 특정 날짜 포맷을 명시하지 않아도 자체적으로 식별해서 데이터를 읽을 수 있음


- 스파크는 두 가지 종류의 시간 관련 정보만 집중적으로 관리함
  - 달력 형태의 날짜(date)
  - 날짜와 시간 정보를 모두 가지는 타임스탬프(timestamp)

  
- TimeStampType클래스는 초 단위 정밀도까지만 지원
  - 밀리세컨드나 마이크로세컨드 단위를 다룬다면 Long 데이터 타입으로 데이터를 변환해서 처리
  - 그 이상의 정밀도는 TimeStampType으로 변환될 때 제거됨
  - 위 같은 문제를 피하려면 파싱이나 변환 작업이 필요

In [0]:
dateDF= spark.range(10).withColumn('today', F.current_date())\
.withColumn('now', F.current_timestamp())

In [0]:
dateDF.createOrReplaceTempView('dateTable')
dateDF.printSchema()

root
-- id: long (nullable = false)
-- today: date (nullable = false)
-- now: timestamp (nullable = false)

### 문자열 -> 날짜 변환

In [0]:
#to_date함수로 문자열 -> 날짜 변환
dateDF.withColumn('birthday', F.to_date(F.lit('1996-07-10'))).show(1)

+---+----------+--------------------+----------+
 id| today| now| birthday|
+---+----------+--------------------+----------+
 0|2021-07-18|2021-07-18 14:54:...|1996-07-10|
+---+----------+--------------------+----------+
only showing top 1 row

In [0]:
#만약 날짜를 파싱할 수 없으면 에러 대신 null값 반환
dateDF.select(F.to_date(F.lit('2021-20-12'))).show(1)

+-------------------+
to_date(2021-20-12)|
+-------------------+
 null|
+-------------------+
only showing top 1 row

### 날짜 연산

In [0]:
#오늘을 기준으로 5일 전후의 날짜 계산
dateDF.select(F.date_sub(F.col('today'),5), F.date_add(F.col('today'),5)).show(1)

+------------------+------------------+
date_sub(today, 5)|date_add(today, 5)|
+------------------+------------------+
 2021-07-13| 2021-07-23|
+------------------+------------------+
only showing top 1 row

In [0]:
#두 날짜 사이의 개월 수 반환
dateDF.select(F.months_between(F.col('today'), F.lit('2020-07-18'))).show(1)

#to_date함수를 적용하지 않아도 자체적으로 식별
# dateDF.select(F.months_between(F.col('today'), F.to_date(F.lit('2020-07-18')))).show(1)

+---------------------------------------+
months_between(today, 2020-07-18, true)|
+---------------------------------------+
 12.0|
+---------------------------------------+
only showing top 1 row

In [0]:
#두 날짜 사이의 일 수 반환
dateDF.select(F.datediff(F.col('today'), F.lit('2020-07-18'))).show(1)

+---------------------------+
datediff(today, 2020-07-18)|
+---------------------------+
 365|
+---------------------------+
only showing top 1 row